# Preparing data for the coherent pipeline
A few steps need to be done on the data before it can be used, namely:
- Take the average over the block - used for later analysis on the CPU
- Multiply the input data the calibration coefficients
- Subtract the continuum
- Sum the 2 polarisations

These are pretty simple. We write the code down here but also do it coherentprep.py

Note: The ordering in memory is not yet decided - there are all sorts of possibilities that we haven't explored yet. it's quite possible hardware gives us ordering that's really weird.

For the sake of this notebook, we put things in (baseline, channel, polarisation, time) order (time going fastest). But as I said, it might well change. The prepare step may well need to do a tranpose or re-ordering of the data on the way through.

In [7]:
%matplotlib inline
import numpy as np
from pylab import *


In [29]:
# Telescope information - more or less fixed
nchan = 288 # number of channels 
nant = 30# number of antennas
nbl = nant*(nant-1)//2 # number of baselines
npol = 2 # number of polariosations
nt = 128 # Number of samples per block
inshape = (nbl, nchan, npol, nt)
dtype = np.dtype('complex64')

print(('input shape={} size={} Mbytes'.format(inshape, np.prod(inshape)*dtype.itemsize/1e6)))


input shape=(435, 288, 2, 128) size=256.57344 Mbytes


In [30]:
def complex_randn(shape, dtype):
    dout = (np.random.randn(*shape) + 1j*np.random.randn(*shape)).astype(dtype)
    return dout

In [31]:
# These data come in from the CPU - note the shapes and type is complex
calibration_data = np.ones((nbl, nchan, npol), dtype=dtype) # all 1 by defult
sky_model = np.zeros((nbl, nchan, npol), dtype=dtype) # 0 by default - polarised sky model
input_data = complex_randn((nbl, nchan, npol, nt), dtype) # make it random for now.


In [32]:
def do_prepare(input_data, calibration_data, sky_model):
    # make output data using numpy broadcasting rules - it's easy to read by doesn't really explain 
    # wht's going on under the hood, but you get the idea
    output_data = input_data * calibration_data[:,:,:,None] - sky_model[:,:,:,None]
    
    # now do the polarisation sum
    output_data = output_data.sum(axis=2)

    # average for cpu - just keep this. Something will copy it back to the CPU when the kernel is finished
    block_average = input_data.mean(axis=3)
    
    return (output_data, block_average)

In [33]:
dout, davg = do_prepare(input_data, calibration_data, sky_model)
assert dout.shape == (nbl, nchan, nt)
assert davg.shape == (nbl, nchan, npol)

In [34]:
%timeit  do_prepare(input_data, calibration_data, sky_model)

KeyboardInterrupt: 

In [ ]:
# This spelled-out look explains the guts of what's going on in more detail.
# It might even go faster too with numba.jit

from numba import jit
#@jit(nopython=True)
def do_prepare_numba(input_data, calibration_data, sky_model, output_data, block_average):
    # make output data using numpy broadcasting rules - cheating slightly, but you get the idea
    nbl, nchan, npol, nt = input_data.shape
    for ibl in range(nbl):
        for ichan in range(nchan):
            for ipol in range(npol):
                caldata = calibration_data[ibl, ichan, ipol]
                skymodel = sky_model[ibl, ichan, ipol ]
                dout = np.zeros(nt, dtype=np.complex)
                blkavg = 0+0j
                for it in range(nt):
                    d = input_data[ibl, ichan, ipol, it] # read input sample
                    blkavg += d                 # block average is the average over all times

                    # Output is the sum over polarisations after applying the calibration and subtracting the sky model
                    # Note these are complex multiplications and sums
                    dout[it] += d * caldata - skymodel # sum over pool
                    
                # stream out average
                block_average[ibl, ichan, ipol] = blkavg
                
            # Stream output data
            for it in range(nt):
                output_data[ibl, ichan, it] = dout[it]
    
    return (output_data, block_average)

In [ ]:
dout.shape
do_prepare_numba(input_data, calibration_data, sky_model, dout, davg)

In [35]:
dout *= 0
davg *= 0

%timeit do_prepare_numba(input_data, calibration_data, sky_model, dout, davg)

KeyboardInterrupt: 

In [36]:
# Wow - numba is 5x faster than naked numpy

In [37]:
# here coherentprep
from craft import coherentprep
import importlib

importlib.reload(coherentprep)
prep = coherentprep.Prepare(nt, calibration_data, sky_model)
dout = prep(input_data)

In [38]:
%timeit prep(input_data)

137 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
